In [1]:
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from string import punctuation
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM, Flatten
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [2]:
df = pd.read_csv('../IMDB_dataset/IMDB dataset.csv')

In [3]:
df.sentiment.replace("positive", 1, inplace = True)
df.sentiment.replace("negative", 0, inplace = True)

In [4]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

x_train, y_train = train['review'], train['sentiment']
x_test, y_test = test['review'], test['sentiment']
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                             test_size=0.2, random_state=10)

In [5]:
tokenizer = Tokenizer(oov_token="OOV")

In [6]:
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [7]:
sequences = tokenizer.texts_to_sequences(x_train)

In [8]:
max_len = 595
trunc_type = 'post'
oov_tok = 'OOV'
padding_type = 'post'
vocab_size = len(word_index) + 1

In [9]:
train_padded = pad_sequences(sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

test_sentences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sentences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

val_sentences = tokenizer.texts_to_sequences(x_val)
val_padded = pad_sequences(val_sentences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [10]:
embedding_dim = 400

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 595, 400)          40637200  
                                                                 
 dropout (Dropout)           (None, 595, 400)          0         
                                                                 
 lstm (LSTM)                 (None, 595, 64)           119040    
                                                                 
 flatten (Flatten)           (None, 38080)             0         
                                                                 
 dense (Dense)               (None, 1)                 38081     
                                                                 
Total params: 40,794,321
Trainable params: 40,794,321
Non-trainable params: 0
_________________________________________________________________


In [13]:
batch_size = 4
num_epochs = 1

In [14]:
history = model.fit(train_padded, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(val_padded, y_val))

8000/8000 [==============================] - 698s 87ms/step - loss: 0.3261 - accuracy: 0.8642 - val_loss: 0.2616 - val_accuracy: 0.8951


In [15]:
model.evaluate(test_padded, y_test)

313/313 [==============================] - 6s 16ms/step - loss: 0.2522 - accuracy: 0.8963


[0.2522154450416565, 0.8963000178337097]

In [16]:
model.save('LSTM_89.51.h5')

In [18]:
import pickle

with open('lstm_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Load Model

In [15]:
review = pd.read_csv('../IMDB_dataset/IMDB dataset.csv')['review'][0]

In [16]:
review

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [17]:
import pickle

tokenizer_filename = './Models/LSTM/lstm_tokenizer.pickle'
tokenizer = pickle.load(open(tokenizer_filename, 'rb'))

model_filename = './Models/LSTM/LSTM_89.51.h5'
model = tf.keras.models.load_model(model_filename)

In [18]:
max_len = 595
trunc_type = 'post'
padding_type = 'post'

In [19]:
import numpy as np

In [20]:
review = tokenizer.texts_to_sequences([review])

In [21]:
review = tf.keras.utils.pad_sequences(review, maxlen=max_len, padding=padding_type, truncating=trunc_type)

review = np.reshape(review, (1, max_len))

In [22]:
prediction = model.predict(review)

1/1 [==============================] - 4s 4s/step


In [26]:
prob_pos = prediction[0][0]

In [27]:
prob_neg = 1 - prediction[0][0]

In [28]:
prob_pos

0.9794652

In [29]:
prob_neg

0.020534813404083252

In [30]:
reviews = pd.read_csv('../IMDB_dataset/IMDB dataset.csv')['review'][0:5]

In [31]:
sequences = tokenizer.texts_to_sequences(reviews)
sequences = tf.keras.utils.pad_sequences(sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
prediction = model.predict(sequences)

1/1 [==============================] - 3s 3s/step


In [34]:
prediction

array([[0.9794652 ],
       [0.9904322 ],
       [0.94077224],
       [0.27642617],
       [0.90864295]], dtype=float32)

In [37]:
scores = [score[0] for score in prediction]

In [38]:
scores

[0.9794652, 0.9904322, 0.94077224, 0.27642617, 0.90864295]

In [39]:
output = []
for score in scores:
    output.append({"prob_pos": "{:.2f}".format(float(score)), "prob_neg": "{:.2f}".format(float(1 - score)) })
output
                   

[{'prob_pos': '0.98', 'prob_neg': '0.02'},
 {'prob_pos': '0.99', 'prob_neg': '0.01'},
 {'prob_pos': '0.94', 'prob_neg': '0.06'},
 {'prob_pos': '0.28', 'prob_neg': '0.72'},
 {'prob_pos': '0.91', 'prob_neg': '0.09'}]